In [268]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from re import sub
from decimal import Decimal

PATH = "./data.insideairbnb.com"
LISTING_FEATURES = ['id', 
                    'price', 
                    'room_type', 
                    'minimum_nights', 
                    'availability_365']

In [303]:
# load static currency factors
CURRENCY_FACTORS = pd.read_excel('currency_factors.xlsx', dtype={'city': str, 'currency': str, 'currency_factor': float})

# define function to convert prices to USD
def assure_usd_price(original_price: str, city: str) -> np.float64:

    if str(original_price).startswith('$') == True:
        factor = 1.0 
    else:
        factor = CURRENCY_FACTORS.loc[(CURRENCY_FACTORS['city'] == city), 'currency_factor']

    #print(type(CURRENCY_FACTORS.loc[(CURRENCY_FACTORS['city'] == city), 'currency_factor']))
    return float(sub(r'[^\d.]', '', str(original_price))) / factor


In [306]:
# read all listing files, and required features
all_listings = pd.DataFrame()
listings_filelist = Path(PATH).rglob("listings.csv")
for file_path in listings_filelist:
    city = file_path.parts[-4]
    city_listing = pd.read_csv(file_path,
                                usecols=LISTING_FEATURES,
                                dtype={'price' : np.float64},
                                index_col='id')
    city_listing['city'] = city
    print(city_listing.dtypes)

    # convert price into USD
    city_listing['price_usd'] = city_listing['price'].apply(lambda x: assure_usd_price(x, city))

    # concat with all
    all_listings = pd.concat([all_listings, city_listing], axis=0)


room_type            object
price               float64
minimum_nights        int64
availability_365      int64
city                 object
dtype: object
float64
room_type            object
price               float64
minimum_nights        int64
availability_365      int64
city                 object
dtype: object
float64


ValueError: Wrong number of items passed 0, placement implies 1

In [146]:
all_listings.head()

,room_type,price,minimum_nights,availability_365,city,price_usd
id,,,,,,
20083,Entire home/apt,1043,4,364,stockholm,[119.06773411784548]
75590,Entire home/apt,949,30,139,stockholm,[108.33679739006266]
155220,Entire home/apt,1200,3,48,stockholm,[136.99068163126995]
164448,Private room,643,3,166,stockholm,[73.40417357408882]
170651,Entire home/apt,634,4,296,stockholm,[72.3767434618543]
